# Elaboración del modelo

En la siguiente celda voy a incluir todo el proceso de limpieza del dataframe que ya hemos visto en clase.

In [1]:
import pandas as pd #permite manipular los dataframes, que serían el equivalente a los Excel
import numpy as np #en caso se necesiten cálculos numéricos, como p. ej. encontrar valores nulos 
from sklearn.model_selection import train_test_split #para dividir datos en entrenamiento y testeo
from sklearn import preprocessing #para escalar/normalizar variables numéricas
import tensorflow as tf #Esta y las de abajo las necesitamos siempre para ANN y RF
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #para calcular parámetros del modelo
import pickle 

In [2]:
base=pd.read_csv('rawdata/habitaclia4043.csv')

#Elimino variables que no aportan o son repetitivas
noimportan=base[['Unnamed: 0', 'OBJECTID_1', 'codigo_inmueble1', 'Title', 'Type_build', 'Type_opera', 
     'Link', 'Location', 'Lat_X', 'Lon_Y', 'Climatic_Z', 'Nom_Mun','Dum_precio','COD_MUN',
     'COD_MUN','precio_eur','Unit_price','Ln_unit_pr',
            'calificacion_emision_valor','ascensor','C_contempo', 'C_estado', 'C_armarios', 'B_contempo', 
            'B_estado', 'B_lavamano', 'R_contempo', 'R_estado', 'R_carpinte', 'R_singular', 
            'R_ventana', 'Precio_red', 'scrap_year','Muestra_2023']]
base=base.drop(noimportan.columns,axis=1)

#Elimino variables no numéricas
no_numericas = base.select_dtypes(exclude=['number']).columns
base=base.drop(no_numericas, axis=1)

base=base.dropna(subset=['Estado_contemporaneidad_calidad','Ausencia_singulares_presencia_arm_cocina'])

base=base.drop(['ratio_bano_hab','Inverse_Age'],axis=1)

X = base.drop('Ln_total_pr', axis=1)  # Todas las columnas excepto 'Ln_total_pr'
y = base['Ln_total_pr']  # Columna 'Ln_total_pr'

# Dividir los datos en conjuntos de entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(
                                                    X, y, 
                                                    test_size=0.25, 
                                                    random_state=42 #Puede ser cualquier número
                                                    )

OHE=X_train[['Year_Before_1981', 'Year_1982_2006', 'Year_After_2007',
       'grand_terr_20m2', 'superficie_jardin_m2', 'bool_despacho',
       'bool_buhardilla', 'bool_trastero', 'bool_lavadero',
       'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado',
       'bool_ascensor', 'bool_aire_acondicionado', 'bool_calefaccion',
       'bool_chimenea', 'Dum_EPC', 'EPC_A_emision', 'EPC_B_emision',
       'EPC_C_emision', 'EPC_D_emision', 'EPC_E_emision', 'EPC_F_emision',
       'EPC_G_emision', 'dum_acces_viappal', 'calidad_cocina', 'diseny_cocina',
       'alta_calidad', 'reform_inmob', 'dum_mar_200m', 'dum_ttpp_riel_urb',
       'EPC_A_emision_2023', 'EPC_B_emision_2023', 'EPC_C_emision_2023',
       'EPC_D_emision_2023', 'EPC_E_emision_2023', 'EPC_F_emision_2023',
       'EPC_G_emision_2023']]

# Crear una lista con todas las columnas binarias
columnas_a_eliminar = list(OHE.columns)

# Eliminar las columnas binarias de X_trainOLS para quedarme sólo con las numéricas
X2 = X_train.drop(columns=columnas_a_eliminar)

#Estandarizamos los valores de X3
min_max_scaler = preprocessing.MinMaxScaler() #preprocessing es una función de la librearía 
                                              #sklearn, ya sabe lo que hacer
X3 = min_max_scaler.fit_transform(X2)

# Convertir OHE a un array NumPy
OHE_array=OHE.values

# Concatenar X3, y OHE_array a lo largo del eje de las columnas (axis=1)
Xtrain_scale = np.concatenate((X3,OHE_array), axis=1)

ytrain_array = y_train.values

OHE2=X_test[['Year_Before_1981', 'Year_1982_2006', 'Year_After_2007',
       'grand_terr_20m2', 'superficie_jardin_m2', 'bool_despacho',
       'bool_buhardilla', 'bool_trastero', 'bool_lavadero',
       'bool_piscina_comunitaria', 'bool_jardin_comunitario', 'bool_amueblado',
       'bool_ascensor', 'bool_aire_acondicionado', 'bool_calefaccion',
       'bool_chimenea', 'Dum_EPC', 'EPC_A_emision', 'EPC_B_emision',
       'EPC_C_emision', 'EPC_D_emision', 'EPC_E_emision', 'EPC_F_emision',
       'EPC_G_emision', 'dum_acces_viappal', 'calidad_cocina', 'diseny_cocina',
       'alta_calidad', 'reform_inmob', 'dum_mar_200m', 'dum_ttpp_riel_urb',
       'EPC_A_emision_2023', 'EPC_B_emision_2023', 'EPC_C_emision_2023',
       'EPC_D_emision_2023', 'EPC_E_emision_2023', 'EPC_F_emision_2023',
       'EPC_G_emision_2023']]

# Crear una lista con todas las columnas binarias
columnas_a_eliminar = list(OHE2.columns)

# Eliminar las columnas binarias de X_testOLS para quedarme sólo con las numéricas
X2_2 = X_test.drop(columns=columnas_a_eliminar)

#Estandarizamos los valores de X3
X3_2 = min_max_scaler.fit_transform(X2_2)

# Convertir OHE a un array NumPy
OHE2_array=OHE2.values

# Concatenar X3, y OHE_array a lo largo del eje de las columnas (axis=1)
Xtest_scale = np.concatenate((X3_2,OHE2_array), axis=1)

# Convierto la Y en array también
ytest_array = y_test.values

Para no volver a repetir todo el proceso de limpieza de datos, voy a guardar los que termino usando, que son `Xtest_scale` y `ytest_array`. De esta forma, para una próxima vez sólo debo 'llamar' a estos arrays (que se guardan en formato de *numpy*). Verán cómo hacer esa llamada en el siguiente cuaderno.

In [3]:
np.save('data/Xtrain_scale.npy', Xtrain_scale)
np.save('data/ytrain_array.npy', ytrain_array)

np.save('data/Xtest_scale.npy', Xtest_scale)
np.save('data/ytest_array.npy', ytest_array)

Ahora estructuro el modelo. No se preocupen por el warning, ya lo he revisado y no es nada de qué preocuparse.

In [4]:
#Añado una semilla de aleatorización
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

#Defino el modelo
model = Sequential([
    Dense(128, activation='relu',
    input_shape=(161,)), #### Noten que acá va la cantidad de X #####
    Dense(64, activation='relu'),
    Dense(1,activation='relu') 
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.005), #Valor usado en la literatura
    loss='mean_squared_error'
)

nnmodel = model.fit(Xtrain_scale, ytrain_array, epochs=100, #Valor usado en la literatura
               batch_size=6, #Valor usado en la literatura
               verbose=False, #Sólo para que no imprima muchas cosas mientras procesa el modelo
               callbacks=[EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)], #Valores usados en la literatura
               validation_split=0.2 #Valor usado en la literatura
              )

C:\Users\paul_\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Reviso el rendimiento del modelo con los datos de prueba.

In [5]:
# Predicciones en el conjunto de testeo
y_predANN = model.predict(Xtest_scale)

# Coefficient of determination (R2)
r2ANN = r2_score(ytest_array, y_predANN)

# Root Mean Squared Error (RMSE)
rmseANN = np.sqrt(mean_squared_error(ytest_array, y_predANN))

# Calcular n y k
n = len(ytest_array)  # Número de observaciones
k = Xtest_scale.shape[1]  # Número de variables independientes

# Calcular R^2 ajustado
r2ANN_adjusted = 1 - ((1 - r2ANN) * (n - 1) / (n - k - 1))

# Imprimir resultados
print("Coefficient of Determination (R2):", r2ANN)
print("Adjusted Coefficient of Determination(R2 adj):", r2ANN_adjusted)
print("Root Mean Squared Error (RMSE):", rmseANN)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Coefficient of Determination (R2): 0.5176558230479482
Adjusted Coefficient of Determination(R2 adj): 0.4209467165855817
Root Mean Squared Error (RMSE): 0.37535636139788636


Ahora guardaré el modelo en el formato *.h5*, que es algo que no tenía el código que vimos en clase. Y el historial en formato *.pickle* (antes estaba con *.pkl*). 

In [6]:
# Guardar el modelo
model.save("model/NN5-BCN4K.h5")

# Guardar el historial
with open("model/history-NN5-BCN4K.pickle", "wb") as f:
    pickle.dump(nnmodel.history, f)

Lo que sugiere el warning es guardar el modelo en formato *.keras* en lugar de *.h5*. Lo guardaré también en ese formato para que vean que el resultado es el mismo.

In [7]:
# Guardar el modelo
model.save("model/NN5-BCN4K.keras")

#Ya no guardo el historial porque el warning no era sobre él.

# Cargo el modelo

<mark>Ahora supongamos lo siguiente</mark>: ya han obtenido el modelo y guardado sus resultados. Sin embargo, han cerrado el Jupyter Lab y al volver a abrirlo quieren cargar los resultados idénticos que obtuvieron antes. 

A continuación les muestro cómo hacerlo, ya sea que hayan guardado su modelo en formato *.h5* o *.keras*. No necesitan guardar el modelo en ambos formatos, yo lo hago sólo para que noten que el resultado es el mismo.

## Archivo *.h5*

Voy a importar el modelo con formato *.h5* en el objeto `loaded_model`. El warning nos indica que debemos construir las métricas, y es lo que buscamos hacer con las siguientes celdas.

In [8]:
from keras.models import load_model

# Cargar el modelo
loaded_model = load_model("model/NN5-BCN4K.h5")

# Cargar el historial
with open("model/history-NN5-BCN4K.pickle", "rb") as f:
    loaded_history = pickle.load(f)

Como no quiero volver a hacer todo el proceso de importación y limpieza de datos, sólo voy a importar aquellos que guardé anteriormente. Ojo, <mark>recuerden que nos estamos poniendo en la situación que he reiniciado el Jupyter Lab y por ello mi PC no 'recuerda' a mis arrays.</mark>

Cargo sólo los datos de prueba porque es sobre ellos que estoy calculando las métricas.

In [9]:
Xtest_scale=np.load('data/Xtest_scale.npy')
ytest_array=np.load('data/ytest_array.npy')

Y ahora puedo ver mis métricas llamando a `loaded_model`.

In [10]:
# Predicciones en el conjunto de testeo
y_predANN = loaded_model.predict(Xtest_scale) #noten que ahora llamo a 'loaded_model'

# Coefficient of determination (R2)
r2ANN = r2_score(ytest_array, y_predANN)

# Root Mean Squared Error (RMSE)
rmseANN = np.sqrt(mean_squared_error(ytest_array, y_predANN))

# Calcular n y k
n = len(ytest_array)  # Número de observaciones
k = Xtest_scale.shape[1]  # Número de variables independientes

# Calcular R^2 ajustado
r2ANN_adjusted = 1 - ((1 - r2ANN) * (n - 1) / (n - k - 1))

# Imprimir resultados
print("Coefficient of Determination (R2):", r2ANN)
print("Adjusted Coefficient of Determination(R2 adj):", r2ANN_adjusted)
print("Root Mean Squared Error (RMSE):", rmseANN)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Coefficient of Determination (R2): 0.5176558230479482
Adjusted Coefficient of Determination(R2 adj): 0.4209467165855817
Root Mean Squared Error (RMSE): 0.37535636139788636


Como notan, son idénticas a las que obtuve inicialmente.

## Archivo *.keras*

Ahora importaré el modelo en el objeto `loaded_model2`. El motivo por el que no me gusta usar este formato (*.keras*) es por el warning: indica que el optimizador (entiendo, el de Adam) está usando menos variables de las que tenía mientras se entrenó. Sin embargo, como verán después, el resultado es exactamente igual.

In [11]:
from keras.models import load_model

# Cargar el modelo
loaded_model2 = load_model("model/NN5-BCN4K.keras") #noten que ahora llamo al archivo .keras, antes era el .h5

# Cargar el historial
with open("model/history-NN5-BCN4K.pickle", "rb") as f:
    loaded_history = pickle.load(f)

C:\Users\paul_\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [12]:
# Predicciones en el conjunto de testeo
y_predANN = loaded_model2.predict(Xtest_scale) #noten que ahora llamo a 'loaded_model'

# Coefficient of determination (R2)
r2ANN = r2_score(ytest_array, y_predANN)

# Root Mean Squared Error (RMSE)
rmseANN = np.sqrt(mean_squared_error(ytest_array, y_predANN))

# Calcular n y k
n = len(ytest_array)  # Número de observaciones
k = Xtest_scale.shape[1]  # Número de variables independientes

# Calcular R^2 ajustado
r2ANN_adjusted = 1 - ((1 - r2ANN) * (n - 1) / (n - k - 1))

# Imprimir resultados
print("Coefficient of Determination (R2):", r2ANN)
print("Adjusted Coefficient of Determination(R2 adj):", r2ANN_adjusted)
print("Root Mean Squared Error (RMSE):", rmseANN)

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Coefficient of Determination (R2): 0.5176558230479482
Adjusted Coefficient of Determination(R2 adj): 0.4209467165855817
Root Mean Squared Error (RMSE): 0.37535636139788636
